# 10Pearls - User Profile Creation Tool 
---

In [ ]:
pip install -r requirements.txt

In [2]:
import os
import tempfile
from docx.shared import Pt
from openai import Client
from docx import Document
from docx.enum.section import WD_SECTION
from dotenv import load_dotenv
from python_docx_replace import docx_replace
import re

### Uploading the File
Keep in mind this works exclusively with old profile formats and not CVs

In [87]:
load_dotenv()
key = os.getenv("OPENAI_API_KEY")

In [88]:
old_doc_path = os.path.join('samples', 'Sheharyar Mohsin - Salesforce (2).docx')
old_doc = Document(old_doc_path)

old_doc_text = ""

for paragraph in old_doc.paragraphs:
    old_doc_text += paragraph.text + "\n"


print(old_doc_text)

Sheharyar Mohsin
A Senior Salesforce Consultant with 7 years of experience in the field of Salesforce CRM and professional experience in software analysis and development. Worked on Salesforce Experience Cloud, Sales Cloud and Service Cloud. Skilled in Lightening Web components and Aura. Experienced with different stages of software development lifecycle. 

Academic Summary:
Master of Science in Computer Science 
Bachelor of Science in Computer Science 

Certification/ Training Summary:
Platform Developer 1 – Salesforce
Administrator – Salesforce
Platform App Builder – Salesforce
Sales Cloud Consultant – Salesforce
Service Cloud Consultant – Salesforce
Sharing and Visibility Architect – Salesforce
Data Architect – Salesforce
Application Architect – Salesforce

Tech Stack/ Skills:
Languages: HTML, CSS, JavaScript, SOQL, Apex
Development: Lightning Web Components, Lightning Aura Components, Visualforce Pages, Force.com sites, Integration with JSON payload, Creating Custom Fields, Validat

In [89]:
job_description = "They have a need of at least 2 to 3 Data Engineers with solid experience in Databricks, and creating and manipulating ETL jobs. Databricks certification will be ideal."

In [67]:
job_description = "Need a Full Stack Developer with MERN expertise in MongoDB, Express JS, React and Node"

### Passing text through OpenAI API 

In [90]:
client = Client(api_key=key)

prompt = f"""Here is the CV/Resume of a person, I want you to efficiently convert the following profile into the new format provided below .
I also need you to compare the following job decription (which is basically a requirement or need for a person) with the person's profile and give me a percentage of how likely the person
is to be a match for the job. I also want you to give me the missing keywords that are present in the job description but not in the person's profile  
Ensure the output is consistently structured as follows:

Job Description:
{job_description}

Old format:
{old_doc_text}

New format:

Resource Name: (Extract and display the profile's Resource Name as the heading)
Percentage Match: (Percentage of how much the person matches the job)
Missing Keywords: (Keywords that are present in job description but not in users profile)
Job Role: (Extract and display the profile's Job Role from the summary as the heading)
Summary: (Keep the summary the same)
Key Skills & Competencies: (Identify and extract at least 6 attributes from the profile and explain them. Examples include "technical expertise", "technical toolkit", "communication and collaboration")
Academic Summary: (Display as provided)
Work Summary: (Provide in detailed paragraphs without altering any information. For each project, include the following details)
   - Project 1, 2 or whatever: (The name of the project, this CAN NOT be empty)
   - Environment: (Always identify and display the job role from the respective project separated by commas)
   - Project Name: (Display under the "Project Name" heading)
   - Outline: (For each project, give the overview of the project )
   - Responsibilities: (For each project, detail the responsibilities separated by commas, do NOT use points)


                     """



response = client.chat.completions.create(
    model='gpt-3.5-turbo-16k',
    messages=[{"role": "assistant", "content": [{"type": "text", "text": prompt}]}],
    max_tokens=3000,
    temperature=0.2,
    top_p=1.0
)

In [91]:
resp = response.choices[0].message.content
print(resp)

Resource Name: Sheharyar Mohsin
Percentage Match: 80%
Missing Keywords: Databricks, ETL jobs, Databricks certification
Job Role: Senior Salesforce Consultant
Summary: A Senior Salesforce Consultant with 7 years of experience in the field of Salesforce CRM and professional experience in software analysis and development. Worked on Salesforce Experience Cloud, Sales Cloud and Service Cloud. Skilled in Lightening Web components and Aura. Experienced with different stages of software development lifecycle.

Key Skills & Competencies:
1. Technical Expertise: Skilled in Salesforce CRM, with experience in Salesforce Experience Cloud, Sales Cloud, and Service Cloud. Proficient in Lightning Web Components and Aura.
2. Technical Toolkit: Experienced in HTML, CSS, JavaScript, SOQL, Apex. Familiar with Lightning Aura Components, Visualforce Pages, and Force.com sites. Knowledgeable in integration with JSON payload and creating custom fields, validation rules, and formula fields.
3. Communication a

### Processing the response 
Removing additional line spaces so that response is always in a fixed format

In [92]:
resp = resp.split("\n")

In [73]:
resp[2]

'Missing Keywords: MERN, MongoDB, Express JS, React, Node'

In [93]:
resp.remove('')

In [75]:
print(resp)

['Resource Name: Muhammad Taha Qadri Rizvi', 'Percentage Match: 80%', 'Missing Keywords: MERN, MongoDB, Express JS, React, Node', 'Job Role: Full Stack Developer', 'Summary: A dynamic professional with 5+ years of experience in analyzing, designing, and developing in an agile environment. Have expertise in JavaScript-based tech stacks such as MERN and Angular on small to large-scale projects. Focused areas include full-stack development and architecture, writing readable code while ensuring quality. Have worked on existing products to provide support, develop new features, and fix the bugs on live products. Have excellent troubleshooting and analytical skills. A well-organized and self-motivated individual with an ability to work well with minimal supervision and meet deadlines.', 'Key Skills & Competencies:', '1. Technical Expertise: Muhammad has 5+ years of experience in analyzing, designing, and developing in an agile environment. He has expertise in JavaScript-based tech stacks suc

In [94]:
text = "\n".join(resp)

# Use regex to extract the required sections
resource_name = re.search(r"Resource Name:\s*(.*)", text).group(1)
job_role = re.search(r"Job Role:\s*(.*)", text).group(1)
percentage_match = re.search(r"Percentage Match:\s*(.*)", text).group(1)
missing_keywords = re.search(r"Missing Keywords:\s*(.*)", text).group(1)
summary = re.search(r"Summary:\s*(.*)", text).group(1)
key_skills_competencies = re.search(r"Key Skills & Competencies:\s*(.*?)Academic Summary:", text, re.DOTALL).group(1).strip()
academic_summary = re.search(r"Academic Summary:\s*(.*?)Work Summary:", text, re.DOTALL).group(1).strip()
work_summary = re.search(r"Work Summary:\s*(.*)", text, re.DOTALL).group(1).strip()



In [95]:
print(work_summary)

Project 1: Retail Business Support and Enhancement Project
Environment: Experience Cloud, Service Cloud, Aura
Outline: The project is a retail business having different products. Salesforce was mainly used for after-sales processes like warranty, refund, replacement, etc. Also includes live agent chat and case management.
Responsibilities: Worked as Senior Salesforce Developer, worked with case management (web-to-case, email-to-case etc.), worked with Lightning Aura components, worked on Custom logic for Apex triggers, carried out custom development with Experience Cloud using Aura components, worked with Record types, page layouts, and lightning pages

Project 2: Real Estate Services
Environment: Service and Experience Cloud, Aura
Outline: A support project for Service and Experience Cloud. This project involved leveraging out-of-the-box functionalities provided by Salesforce to implement client requirements of Lead assignment rules, Case assignment rules, and Community customizations

In [96]:
# Store in appropriate data structures
profile_data = {
    "Resource Name": resource_name,
    "Job Role": job_role,
    "Percentage Match": percentage_match,
    "Missing Keywords": missing_keywords,
    "Summary": summary,
    "Key Skills & Competencies": key_skills_competencies.split("\n"),
    "Academic Summary": academic_summary.split("\n"),
    "Work Summary": []
}

# Extract individual projects in Work Summary
projects = re.findall(
    r"Project \d+: (.*?)\nEnvironment:\s*(.*?)\nOutline:\s*(.*?)\nResponsibilities:\s*(.*?)(?:\n\n|$)",
    work_summary, re.DOTALL
)

# Populate profile_data with extracted projects
for project in projects:
    project_data = {
        "Project": project[0],
        "Environment": project[1],
        "Outline": project[2].strip(),
        "Responsibilities": project[3].strip()
    }
    profile_data["Work Summary"].append(project_data)

# Print extracted data
for key, value in profile_data.items():
    print(f"{key}: {value}")

Resource Name: Sheharyar Mohsin
Job Role: Senior Salesforce Consultant
Percentage Match: 80%
Missing Keywords: Databricks, ETL jobs, Databricks certification
Summary: A Senior Salesforce Consultant with 7 years of experience in the field of Salesforce CRM and professional experience in software analysis and development. Worked on Salesforce Experience Cloud, Sales Cloud and Service Cloud. Skilled in Lightening Web components and Aura. Experienced with different stages of software development lifecycle.
Key Skills & Competencies: ['1. Technical Expertise: Skilled in Salesforce CRM, with experience in Salesforce Experience Cloud, Sales Cloud, and Service Cloud. Proficient in Lightning Web Components and Aura.', '2. Technical Toolkit: Experienced in HTML, CSS, JavaScript, SOQL, Apex. Familiar with Lightning Aura Components, Visualforce Pages, and Force.com sites. Knowledgeable in integration with JSON payload and creating custom fields, validation rules, and formula fields.', '3. Communic

In [97]:
print(profile_data['Work Summary'][0]['Project'])

Retail Business Support and Enhancement Project


### Generating Word Document
Will include everything according to the provided format except for "Percentage Match" and "Missing Keywords"

In [98]:
from docx.shared import Pt
from docx.oxml.ns import qn

In [99]:
new_file_path = os.path.join('template', 'Sample Profile 2.docx')
new_file = Document(new_file_path)

In [100]:
def set_font_style(document, font_name='Arial'):
    for paragraph in document.paragraphs:
        for run in paragraph.runs:
            run.font.name = font_name
            run._element.rPr.rFonts.set(qn('w:eastAsia'), font_name)  # For East Asian text support


def replace_placeholder(paragraph, placeholder, replacement):
    if placeholder in paragraph.text:
        combined_text = "".join([run.text for run in paragraph.runs])
        new_text = combined_text.replace(placeholder, replacement)
        
        # Clear existing runs and add the replaced text as a single run
        for run in paragraph.runs:
            run.text = ""
        paragraph.runs[0].text = new_text

# Function to add bold text
def bold_text(paragraph, text):
    run = paragraph.add_run(text)
    run.bold = True

In [101]:
for paragraph in new_file.paragraphs:
    replace_placeholder(paragraph, "${resourcename}", profile_data["Resource Name"])
    replace_placeholder(paragraph, "${jobrole}", profile_data["Job Role"])
    replace_placeholder(paragraph, "${summary}", profile_data["Summary"])
    replace_placeholder(paragraph, "${academic}", "\n".join(profile_data["Academic Summary"]))


In [102]:
for paragraph in new_file.paragraphs:
    if "${keyskills}" in paragraph.text:
        paragraph.clear()
        for skill in profile_data["Key Skills & Competencies"]:
            skill_parts = skill.split(":")
            
            if len(skill_parts) == 2:
                bold_text(paragraph, skill_parts[0] + ": ")
                competencies = skill_parts[1].split(", ")
                for competency in competencies:
                    competency = competency.strip()
                    paragraph.add_run("\n   • ")
                    bullet_point = paragraph.add_run(competency)
                    bullet_point.font.size = Pt(11)
                    
                paragraph.add_run("\n")
                paragraph.add_run("\n")  # Add new line after each skill section
        

In [103]:
# Replace Work summary
for paragraph in new_file.paragraphs:
    if "${worksummary}" in paragraph.text:
        paragraph.clear()
        for project in profile_data["Work Summary"]:
            project_paragraph = new_file.add_paragraph()
            bold_text(project_paragraph, "Project: ")
            project_paragraph.add_run(project["Project"])
            new_file.add_paragraph(f"Environment: {project['Environment']}")
            new_file.add_paragraph(f"Outline: {project['Outline']}")
            new_file.add_paragraph(f"Responsibilities: {project['Responsibilities']}")
            new_file.add_paragraph("\n")  # Add new line after each project

In [104]:
set_font_style(new_file, font_name='Arial')

new_file.save('NewSampleFormatCV.docx')
print(f"Document saved")

Document saved
